In [1]:
import json
import numpy as np
import scipy as scs
import pandas as pd
from pandas.io.json import json_normalize 

print("Starting...")

data = json.load(open('formatted_tweets.json'))
data = data["tweets"]
json_normalize(data)
tweets_dataset = pd.DataFrame(data)

print("starting...")

Starting...
starting...


In [2]:
df = pd.DataFrame(columns=['article_id', 'fake','tweet'])
i=0
cleaned_tweets = []

with open("twitter.txt") as myfile:
    for line in myfile:
        cleaned_tweets = []
        line = line.replace('\t', ' ')
        line = line.split()
        fake = ''
        for el in line:
            if el.isdigit():
                cleaned_tweets+=[el]
            elif 'label:' in el:
                fake = el
        df.loc[i] = [i, fake, cleaned_tweets]
        i+=1

In [3]:
df["fake"] = df["fake"].apply(lambda x: False if x.strip() == "label:0" else True)

In [4]:
#much faster
s = df.apply(lambda x: pd.Series(x['tweet']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'tweet'
df = df.drop('tweet', axis=1).join(s)
df = df.reset_index(drop=True)
df.head()

,article_id,fake,tweet
0,0,False,677641866930757635
1,0,False,677644704100544513
2,0,False,677653639389962241
3,0,False,677664025346449409
4,0,False,677682035838861313


In [5]:
tweets_dataset.head()

,created_at,favorite_count,hash_tags,id,in_reply_to_status_id,possibly_sensitive,retweet_count,tweet_text,user_id,user_mentions
0,Tue Jun 30 13:02:48 +0000 2015,0,[],615868085090676737,NULL,False,0,"Walmart bakes ISIS cake, rejects Confederate f...",1643045892,[]
1,Mon May 24 12:23:36 +0000 2010,0,[],14619605813,NULL,False,0,UK bans doctor who linked autism to vaccine: B...,124779476,[]
2,Fri Sep 18 14:07:27 +0000 2015,6,[],644875384622829568,NULL,False,3,Doritos unveils rainbow chips to support LGBT ...,22970986,[]
3,Fri Sep 25 18:52:43 +0000 2015,0,[],647483888684498944,NULL,False,1,Mike Huckabee Is Having A Meltdown About Those...,1179710990,[]
4,Wed Jun 10 17:06:59 +0000 2015,0,[],608681780812652544,NULL,False,0,The only good news about the McKinney pool par...,2234826078,[2347049341]


In [60]:
final_df = df.join(tweets_dataset, lsuffix='tweet', rsuffix='id', how='inner')

In [31]:
len(final_df)

539607

In [61]:
final_df[final_df["user_id"].isnull()]

,article_id,fake,tweet,created_at,favorite_count,hash_tags,id,in_reply_to_status_id,possibly_sensitive,retweet_count,tweet_text,user_id,user_mentions


In [32]:
import csv
final_df.to_csv("complete_table.csv", encoding='utf-8', index=False, sep=';')

In [46]:
import scipy.sparse as sps
from scipy import sparse
arr_articles = final_df["article_id"].values.astype(np.int)
arr_users = final_df["user_id"].values.astype(np.int)
ones = np.ones(len(arr_users))
matrix = sps.coo_matrix((ones, (arr_users, arr_articles)))

In [45]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
reduction = svd.transform(matrix)

KeyboardInterrupt: 

In [62]:
list_id = sorted(set(final_df["id"]))
final_df["new_id"] = final_df["id"].apply(lambda x: list_id.index(x))

print("done...")



done...


NameError: name 'final_dfdf' is not defined

In [50]:
len(final_df["user_id"].drop_duplicates())

221446

In [63]:
list_id = sorted(set(final_df["user_id"]))
final_df["new_user_id"] = final_df["user_id"].apply(lambda x: list_id.index(x))
print("done...")

done...


In [54]:
len(final_df[final_df["user_id"] == "1643045892"])

15

In [57]:
len(final_df.groupby(["user_id"]).filter(lambda x: len(x) > 1)["user_id"].drop_duplicates())

62708

In [65]:
final_df.to_csv("complete_table_end_comma.csv", encoding='utf-8', index=False, sep=',')

In [59]:
final_df.head()

,article_id,fake,tweet,created_at,favorite_count,hash_tags,id,in_reply_to_status_id,possibly_sensitive,retweet_count,tweet_text,user_id,user_mentions
0,0,False,677641866930757635,Tue Jun 30 13:02:48 +0000 2015,0,[],615868085090676737,NULL,False,0,"Walmart bakes ISIS cake, rejects Confederate f...",1643045892,[]
1,0,False,677644704100544513,Mon May 24 12:23:36 +0000 2010,0,[],14619605813,NULL,False,0,UK bans doctor who linked autism to vaccine: B...,124779476,[]
2,0,False,677653639389962241,Fri Sep 18 14:07:27 +0000 2015,6,[],644875384622829568,NULL,False,3,Doritos unveils rainbow chips to support LGBT ...,22970986,[]
3,0,False,677664025346449409,Fri Sep 25 18:52:43 +0000 2015,0,[],647483888684498944,NULL,False,1,Mike Huckabee Is Having A Meltdown About Those...,1179710990,[]
4,0,False,677682035838861313,Wed Jun 10 17:06:59 +0000 2015,0,[],608681780812652544,NULL,False,0,The only good news about the McKinney pool par...,2234826078,[2347049341]
